In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

In [2]:
df = pd.read_csv('dataset/bandgap.csv')
df

,avg_Density_(g/mL),avg_Gordy_electonegativity,avg_Number_of_unfilled_d_valence_electrons,avg_abs_valence,avg_metallic_valence,avg_valence_d,sum_Density_(g/mL),sum_heat_atomization(kJ/mol),sum_valence_d,range_Density_(g/mL),range_Nagle_electonegativity,range_heat_of_fusion_(kJ/mol)_,target
0,4.290000,5.316140,7.000000,2.700000,2.532000,3.000000,42.900,2694.0,30.0,6.48,1.40,9.340,1.220000
1,2.820625,5.087500,9.500000,3.500000,2.250000,0.500000,90.260,9762.0,16.0,7.71,1.65,24.680,1.770000
2,4.810000,4.996168,7.894737,2.210526,2.829474,2.105263,91.390,5247.0,40.0,8.10,1.65,11.330,2.430000
3,8.621500,5.020180,0.250000,3.150000,2.387000,9.750000,17.243,401.5,19.5,5.16,0.72,12.691,0.490000
4,3.490000,5.058571,7.142857,2.857143,2.348571,2.857143,24.430,1747.0,20.0,5.78,1.66,5.310,2.700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2478,5.960000,4.755000,5.000000,2.000000,3.220000,5.000000,11.920,358.0,10.0,2.34,0.85,0.630,2.775131
2479,3.050000,6.494114,7.000000,2.857143,1.491429,3.000000,21.350,1757.0,21.0,9.75,2.51,11.180,1.580000
2480,4.882500,4.680000,8.500000,2.500000,3.000000,1.500000,19.530,2076.0,6.0,6.51,2.45,16.680,0.200000
2481,4.380000,4.535000,9.000000,2.000000,4.000000,1.000000,8.760,1326.0,2.0,4.26,1.38,88.100,0.600000


In [3]:
from hyperimpute.plugins.imputers import Imputers, ImputerPlugin
from Impute import fill_with_et

imputers = Imputers()


class EtImputer(ImputerPlugin):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._model = fill_with_et

    @staticmethod
    def name():
        return "et"

    @staticmethod
    def hyperparameter_space():
        return []

    def _fit(self, *args, **kwargs):
        return self

    def _transform(self, df):
        # 按照缺失值的比例进行排序
        miss_rate = df.isnull().sum() / df.shape[0]
        cols = miss_rate.sort_values().index.tolist()
        cols = [col for col in cols if miss_rate[col] > 0]
        for col in cols:
            df_col_filled = self._model(df, col)
            df[col] = df_col_filled[col]
        return df


imputers.add("et", EtImputer)

hyper = imputers.get("hyperimpute", n_inner_iter=1)
et = imputers.get("et")
missforest = imputers.get("missforest", max_iter=1)
gain = imputers.get("gain", n_epochs=10)
sinkhorn = imputers.get("sinkhorn", n_epochs=10)
mean = imputers.get("mean")

d:\software\miniconda\Lib\site-packages\pydantic\_internal\_fields.py:149: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [4]:
target_col = "target"

In [5]:
# from sklearn.model_selection import StratifiedKFold, KFold
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.linear_model import LinearRegression, Ridge, Lasso
# from hyperimpute.plugins.utils.simulate import simulate_nan
# from sklearn.metrics import r2_score, mean_absolute_error as MAE
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import scienceplots
# import warnings

# warnings.filterwarnings("ignore")

# plt.style.use(['science','no-latex'])
# # set font as times new roman
# plt.rcParams["font.family"] = "Times New Roman"
# plt.figure(figsize=(12, 8))

# X = df.drop([target_col], axis=1)
# y = df[target_col]

# kf = KFold(n_splits=5, shuffle=True, random_state=0)


# scores_et = []
# r2_scores_et = []

# scores_hyper = []
# r2_scores_hyper = []

# scores_missforest = []
# r2_scores_missforest = []

# scores_ori = []
# r2_scores_ori = []

# scores_gain = []
# r2_scores_gain = []

# scores_sinkhorn = []
# r2_scores_sinkhorn = []

# scores_mean = []
# r2_scores_mean = []

# for i, (train_idx, test_idx) in enumerate(kf.split(X, y)):
#     X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
#     y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
#     df_train = pd.concat([X_train, y_train], axis=1)
#     df_test = pd.concat([X_test, y_test], axis=1)
#     # reset the index
#     df_train = df_train.reset_index(drop=True)
#     df_test = df_test.reset_index(drop=True)
#     X_train = df_train.drop([target_col], axis=1)
#     y_train = df_train[target_col]
#     X_test = df_test.drop([target_col], axis=1)
#     y_test = df_test[target_col]
#     cols = X_train.columns

#     X_train = X_train.to_numpy()
#     ret = simulate_nan(X_train, 0.1, mecha="MAR")
#     X_train_null = ret['X_incomp']
#     mask = ret['mask']
#     X_train_null = pd.DataFrame(X_train_null, columns=cols)

#     X_imputed_hyper = hyper.fit_transform(X_train_null.copy())
#     X_imputed_missforest = missforest.fit_transform(X_train_null.copy())
#     X_imputed_et = et.fit_transform(X_train_null.copy())
#     X_imputed_gain = gain.fit_transform(X_train_null.copy())
#     X_imputed_sinkhorn = sinkhorn.fit_transform(X_train_null.copy())
#     X_imputed_mean = mean.fit_transform(X_train_null.copy())

#     rf = RandomForestRegressor(n_estimators=500)
#     rf.fit(X_imputed_hyper, y_train)
#     y_pred_hyper = rf.predict(X_test)
#     hyper_score = MAE(y_test, y_pred_hyper)
#     r2_hyper = r2_score(y_test, y_pred_hyper)
#     print("Hyperimpute score: ", hyper_score)

#     rf = RandomForestRegressor(n_estimators=500)
#     rf.fit(X_imputed_missforest, y_train)
#     y_pred_missforest = rf.predict(X_test)
#     missforest_score = MAE(y_test, y_pred_missforest)
#     r2_missforest = r2_score(y_test, y_pred_missforest)
#     print("Missforest score: ", missforest_score)

#     rf = RandomForestRegressor(n_estimators=500)
#     rf.fit(X_imputed_et, y_train)
#     y_pred_et = rf.predict(X_test)
#     et_score = MAE(y_test, y_pred_et)
#     r2_et = r2_score(y_test, y_pred_et)
#     print("ET score: ", et_score)

#     rf = RandomForestRegressor(n_estimators=500)
#     rf.fit(X_train, y_train)
#     y_pred = rf.predict(X_test)
#     ori_score = MAE(y_test, y_pred)
#     r2_ori = r2_score(y_test, y_pred)
#     print("Original score: ", ori_score)

#     rf = RandomForestRegressor(n_estimators=500)
#     rf.fit(X_imputed_gain, y_train)
#     y_pred_gain = rf.predict(X_test)
#     gain_score = MAE(y_test, y_pred_gain)
#     r2_gain = r2_score(y_test, y_pred_gain)
#     print("Gain score: ", gain_score)

#     rf = RandomForestRegressor(n_estimators=500)
#     rf.fit(X_imputed_sinkhorn, y_train)
#     y_pred_sinkhorn = rf.predict(X_test)
#     sinkhorn_score = MAE(y_test, y_pred_sinkhorn)
#     r2_sinkhorn = r2_score(y_test, y_pred_sinkhorn)
#     print("Sinkhorn score: ", sinkhorn_score)

#     rf = RandomForestRegressor(n_estimators=500)
#     rf.fit(X_imputed_mean, y_train)
#     y_pred_mean = rf.predict(X_test)
#     mean_score = MAE(y_test, y_pred_mean)
#     r2_mean = r2_score(y_test, y_pred_mean)
#     print("Mean score: ", mean_score)

#     scores_et.append(et_score)
#     scores_hyper.append(hyper_score)
#     scores_missforest.append(missforest_score)
#     scores_gain.append(gain_score)
#     scores_sinkhorn.append(sinkhorn_score)
#     scores_mean.append(mean_score)
#     scores_ori.append(ori_score)

#     r2_scores_et.append(r2_et)
#     r2_scores_hyper.append(r2_hyper)
#     r2_scores_missforest.append(r2_missforest)
#     r2_scores_gain.append(r2_gain)
#     r2_scores_sinkhorn.append(r2_sinkhorn)
#     r2_scores_mean.append(r2_mean)
#     r2_scores_ori.append(r2_ori)

#     # plot
#     sns.scatterplot(x=y_test, y=y_pred, s=120, marker='o', linewidths=2, facecolors='#7ED9D9',
#                 edgecolors='#7ED9D9')

#     sns.scatterplot(x=y_test, y=y_pred_hyper,  marker='v', s=150, linewidths=2,
#                 facecolors='none', edgecolors='#9467BD')
#     sns.scatterplot(x=y_test, y=y_pred_missforest, marker='^', s=150,linewidths=2,
#                 facecolors='none', edgecolors='#B3DE69')
#     sns.scatterplot(x=y_test, y=y_pred_gain, marker='>', s=150,linewidths=2,
#                 facecolors='none', edgecolors='#FFC0D9')
#     sns.scatterplot(x=y_test, y=y_pred_sinkhorn,  marker='<', s=150,linewidths=2,
#                  facecolors='none', edgecolors='#5FBDFF')
#     sns.scatterplot(x=y_test, y=y_pred_mean, marker='s', s=150,linewidths=2,
#                 facecolors='none',edgecolors='#FDBF6F')
#     sns.scatterplot(x=y_test, y=y_pred_et, marker='*', s=300, linewidths=0,
#                 color='#F35F5F', edgecolors='#F35F5F')
#     print("=====================================")

# plt.legend(["Original", "HyperImpute", "MissForest", "Gain", "Sinkhorn", "Mean", "MatImpute"], loc='upper left',
#            fontsize=20, frameon=False, bbox_to_anchor=(0, 0.95))

# # plt.legend(['Original: MAE={:.2f}, $R^2$={:.2f}'.format(np.mean(scores_ori), np.mean(r2_scores_ori)),
# #             'MatImpute: MAE={:.2f}, $R^2$={:.2f}'.format(np.mean(scores_et), np.mean(r2_scores_et)),
# #             'HyperImpute: MAE={:.2f}, $R^2$={:.2f}'.format(np.mean(scores_hyper), np.mean(r2_scores_hyper)),
# #             'MissForest: MAE={:.2f}, $R^2$={:.2f}'.format(np.mean(scores_missforest), np.mean(r2_scores_missforest)),
# #             'Gain: MAE={:.2f}, $R^2$={:.2f}'.format(np.mean(scores_gain), np.mean(r2_scores_gain)),
# #             'Sinkhorn: MAE={:.2f}, $R^2$={:.2f}'.format(np.mean(scores_sinkhorn), np.mean(r2_scores_sinkhorn)),
# #             'Mean: MAE={:.2f}, $R^2$={:.2f}'.format(np.mean(scores_mean), np.mean(r2_scores_mean))], loc='upper left',
# #            fontsize=16)

# plt.plot([np.min(y), np.max(y)], [np.min(y), np.max(y)], 'k--', lw=2, color='#161A30')
# plt.xlabel('True Values', fontsize=24)
# plt.ylabel('Predict Values', fontsize=24)
# plt.xticks(fontsize=24)
# plt.yticks(fontsize=24)
# plt.tight_layout()
# plt.savefig('results/pipeline-regression-bandgap.png', dpi=300)

In [6]:
# print("Hyperimpute score:{:.2f}±{:.2f}".format(np.mean(scores_hyper), np.std(scores_hyper)))
# print("Missforest score:{:.2f}±{:.2f}".format(np.mean(scores_missforest), np.std(scores_missforest)))
# print("ET score:{:.2f}±{:.2f}".format(np.mean(scores_et), np.std(scores_et)))
# print("Original score:{:.2f}±{:.2f}".format(np.mean(scores_ori), np.std(scores_ori)))
# print("Gain score:{:.2f}±{:.2f}".format(np.mean(scores_gain), np.std(scores_gain)))
# print("Sinkhorn score:{:.2f}±{:.2f}".format(np.mean(scores_sinkhorn), np.std(scores_sinkhorn)))
# print("Mean score:{:.2f}±{:.2f}".format(np.mean(scores_mean), np.std(scores_mean)))

In [7]:
# print("Hyperimpute r2:{:.2f}±{:.2f}".format(np.mean(r2_scores_hyper), np.std(r2_scores_hyper)))
# print("Missforest r2:{:.2f}±{:.2f}".format(np.mean(r2_scores_missforest), np.std(r2_scores_missforest)))
# print("ET r2:{:.2f}±{:.2f}".format(np.mean(r2_scores_et), np.std(r2_scores_et)))
# print("Original r2:{:.2f}±{:.2f}".format(np.mean(r2_scores_ori), np.std(r2_scores_ori)))
# print("Gain r2:{:.2f}±{:.2f}".format(np.mean(r2_scores_gain), np.std(r2_scores_gain)))
# print("Sinkhorn r2:{:.2f}±{:.2f}".format(np.mean(r2_scores_sinkhorn), np.std(r2_scores_sinkhorn)))
# print("Mean r2:{:.2f}±{:.2f}".format(np.mean(r2_scores_mean), np.std(r2_scores_mean)))

In [8]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.ensemble import RandomForestRegressor
from hyperimpute.plugins.utils.simulate import simulate_nan
from sklearn.metrics import r2_score, mean_absolute_error as MAE
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scienceplots
import warnings
from tqdm import tqdm

warnings.filterwarnings("ignore")

X = df.drop([target_col], axis=1)
y = df[target_col]

n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=0)

original_scores = []
matimpute_scores = []
hyperimpute_scores = []
missforest_scores = []
gain_scores = []
sinkhorn_scores = []
mean_scores = []

for ratio in tqdm([0.1, 0.2, 0.3, 0.4, 0.5]):
    original_score = []
    matimpute_score = []
    hyperimpute_score = []
    missforest_score = []
    gain_score = []
    sinkhorn_score = []
    mean_score = []

    for i, (train_idx, test_idx) in enumerate(kf.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        df_train = pd.concat([X_train, y_train], axis=1)
        df_test = pd.concat([X_test, y_test], axis=1)
        # reset the index
        df_train = df_train.reset_index(drop=True)
        df_test = df_test.reset_index(drop=True)
        X_train = df_train.drop([target_col], axis=1)
        y_train = df_train[target_col]
        X_test = df_test.drop([target_col], axis=1)
        y_test = df_test[target_col]


        cols = X_train.columns
        X_train = X_train.to_numpy()
        ret = simulate_nan(X_train, ratio)
        X_train_null = ret['X_incomp']
        mask = ret['mask']
        X_train_null = pd.DataFrame(X_train_null, columns=cols)

        X_imputed_hyper = hyper.fit_transform(X_train_null.copy())
        X_imputed_missforest = missforest.fit_transform(X_train_null.copy())
        X_imputed_et = et.fit_transform(X_train_null.copy())
        X_imputed_gain = gain.fit_transform(X_train_null.copy())
        X_imputed_sinkhorn = sinkhorn.fit_transform(X_train_null.copy())
        X_imputed_mean = mean.fit_transform(X_train_null.copy())
        
        rf = RandomForestRegressor(n_estimators=500)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)
        original_score.append(r2_score(y_test, y_pred))

        rf = RandomForestRegressor(n_estimators=500)
        rf.fit(X_imputed_et, y_train)
        y_pred_et = rf.predict(X_test)
        matimpute_score.append(r2_score(y_test, y_pred_et))

        rf = RandomForestRegressor(n_estimators=500)
        rf.fit(X_imputed_hyper, y_train)
        y_pred_hyper = rf.predict(X_test)
        hyperimpute_score.append(r2_score(y_test, y_pred_hyper))

        rf = RandomForestRegressor(n_estimators=500)
        rf.fit(X_imputed_missforest, y_train)
        y_pred_missforest = rf.predict(X_test)
        missforest_score.append(r2_score(y_test, y_pred_missforest))

        rf = RandomForestRegressor(n_estimators=500)
        rf.fit(X_imputed_gain, y_train)
        y_pred_gain = rf.predict(X_test)
        gain_score.append(r2_score(y_test, y_pred_gain))

        rf = RandomForestRegressor(n_estimators=500)
        rf.fit(X_imputed_sinkhorn, y_train)
        y_pred_sinkhorn = rf.predict(X_test)
        sinkhorn_score.append(r2_score(y_test, y_pred_sinkhorn))

        rf = RandomForestRegressor(n_estimators=500)
        rf.fit(X_imputed_mean, y_train)
        y_pred_mean = rf.predict(X_test)
        mean_score.append(r2_score(y_test, y_pred_mean))
    original_scores.append(original_score)
    matimpute_scores.append(matimpute_score)
    hyperimpute_scores.append(hyperimpute_score)
    missforest_scores.append(missforest_score)
    gain_scores.append(gain_score)
    sinkhorn_scores.append(sinkhorn_score)
    mean_scores.append(mean_score)

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [40:12<00:00, 482.59s/it]


In [9]:
# save the results

results = [
    original_scores,
    matimpute_scores,
    hyperimpute_scores,
    missforest_scores,
    gain_scores,
    sinkhorn_scores,
    mean_scores
]

# save to npy file
np.save("results/result_bandgap.npy", results)

In [10]:
# # plot the results
# import scienceplots

# plt.style.use(['nature'])
# # set font as times new roman
# plt.rcParams["font.family"] = "Times New Roman"
# plt.figure(figsize=(8, 6))

# # set no grid
# plt.grid(False)

# sns.lineplot(x=[0.1, 0.2, 0.3, 0.4, 0.5], y=original_scores, label='Original', marker='o',color='#7ED9D9',lw=2, markersize=8)
# sns.lineplot(x=[0.1, 0.2, 0.3, 0.4, 0.5], y=matimpute_scores, label='MatImpute', marker='*',color='#F35F5F',lw=2, markersize=12)
# sns.lineplot(x=[0.1, 0.2, 0.3, 0.4, 0.5], y=hyperimpute_scores, label='HyperImpute', marker='v',color='#9467BD',lw=2, markersize=8)
# sns.lineplot(x=[0.1, 0.2, 0.3, 0.4, 0.5], y=missforest_scores, label='MissForest', marker='^',color='#B3DE69',lw=2, markersize=8)
# sns.lineplot(x=[0.1, 0.2, 0.3, 0.4, 0.5], y=gain_scores, label='Gain', marker='>',color='#FFC0D9',lw=2, markersize=8)
# sns.lineplot(x=[0.1, 0.2, 0.3, 0.4, 0.5], y=sinkhorn_scores, label='Sinkhorn', marker='<',color='#5FBDFF',lw=2, markersize=8)
# sns.lineplot(x=[0.1, 0.2, 0.3, 0.4, 0.5], y=mean_scores, label='Mean', marker='s',color='#FDBF6E',lw=2, markersize=7)

# plt.xlabel('Missing Ratio', fontsize=24)
# plt.ylabel('$R^2$ Score',fontsize=24)
# plt.tick_params(labelsize=24)
# legend = plt.legend(fontsize=16, frameon=False, loc='upper right')
# plt.legend().set_visible(False)
# plt.tight_layout(pad=1.5)
# plt.savefig(f'results/missing-ratio-Bandgap.png', dpi=300)

In [11]:
# # get the legend and save to another file

# fig = plt.figure(figsize=(8, 6))
# # set legend in one row
# legend = fig.legend(handles=legend.legendHandles, labels=['Original', 'MatImpute', 'HyperImpute', 'MissForest', 'Gain', 'Sinkhorn', 'Mean'], loc='center', fontsize=24, frameon=False,ncol=7)
# # save to svg
# plt.savefig('results/missing-ratio-Bandgap-legend.svg', dpi=300)